In [ ]:
import boto3
import pandas as pd
from io import BytesIO
import os
from dotenv import load_dotenv

load_dotenv()

In [3]:
# Konfiguracja połączenia
session = boto3.session.Session()
client = session.client('s3',
    region_name=os.getenv('DO_SPACES_REGION'),
    endpoint_url=os.getenv('DO_SPACES_ENDPOINT'),
    aws_access_key_id=os.getenv('DO_SPACES_KEY'),
    aws_secret_access_key=os.getenv('DO_SPACES_SECRET')
)

BUCKET_NAME = os.getenv('DO_SPACES_BUCKET')

# Wczytaj pliki CSV
obj_2023 = client.get_object(Bucket=BUCKET_NAME, Key='data/raw/halfmarathon_2023.csv')
df_2023 = pd.read_csv(BytesIO(obj_2023['Body'].read()), sep=';')

obj_2024 = client.get_object(Bucket=BUCKET_NAME, Key='data/raw/halfmarathon_2024.csv')
df_2024 = pd.read_csv(BytesIO(obj_2024['Body'].read()), sep=';')

# Połącz dane
df = pd.concat([df_2023, df_2024], ignore_index=True)

print(f"Wczytano {len(df)} rekordów")
print(df.head())
print(df.info())

Wczytano 21957 rekordów
   Miejsce  Numer startowy       Imię       Nazwisko   Miasto Kraj  \
0      1.0            1787     TOMASZ         GRYCKO      NaN  POL   
1      2.0               3  ARKADIUSZ  GARDZIELEWSKI  WROCŁAW  POL   
2      3.0            3832  KRZYSZTOF          HADAS   POZNAŃ  POL   
3      4.0             416     DAMIAN         DYDUCH    KĘPNO  POL   
4      5.0            8476      KAMIL      MAŃKOWSKI   MIRKÓW  POL   

                     Drużyna Płeć  Płeć Miejsce Kategoria wiekowa  ...  \
0     UKS BLIZA WŁADYSŁAWOWO    M           1.0               M30  ...   
1  ARKADIUSZGARDZIELEWSKI.PL    M           2.0               M30  ...   
2                        NaN    M           3.0               M20  ...   
3   AZS POLITECHNIKA OPOLSKA    M           4.0               M30  ...   
4            PARKRUN WROCŁAW    M           5.0               M20  ...   

   10 km Tempo  15 km Czas 15 km Miejsce Open  15 km Tempo  20 km Czas  \
0     2.926667    00:44:47          

In [4]:
import numpy as np
from datetime import datetime

# Kopia danych do czyszczenia
df_clean = df.copy()

# 1. Normalizacja płci (M, K)
df_clean['sex'] = df_clean['Płeć'].str.upper().str.strip()
df_clean = df_clean[df_clean['sex'].isin(['M', 'K'])]

# 2. Konwersja Rocznik → age (rok bieżący zależy od danych)
current_year_2023 = 2023
current_year_2024 = 2024
# Sprawdź z którego roku są dane na podstawie pierwszego wiersza lub dodaj kolumnę year
df_clean['age'] = np.where(df_clean['Rocznik'].notna(), 
                            current_year_2024 - df_clean['Rocznik'], 
                            np.nan)

# 3. Konwersja czasów na sekundy
def time_to_seconds(time_str):
    if pd.isna(time_str):
        return np.nan
    try:
        parts = time_str.strip().split(':')
        if len(parts) == 3:  # HH:MM:SS
            return int(parts[0]) * 3600 + int(parts[1]) * 60 + int(parts[2])
        elif len(parts) == 2:  # MM:SS
            return int(parts[0]) * 60 + int(parts[1])
        return np.nan
    except:
        return np.nan

df_clean['time_5km_s'] = df_clean['5 km Czas'].apply(time_to_seconds)
df_clean['time_half_s'] = df_clean['Czas'].apply(time_to_seconds)

# 4. Odrzucenie rekordów z brakami w kluczowych kolumnach
df_clean = df_clean.dropna(subset=['sex', 'age', 'time_5km_s', 'time_half_s'])

# 5. Sanity check: czas 5 km * 4 < czas półmaratonu
df_clean = df_clean[df_clean['time_5km_s'] * 4 < df_clean['time_half_s']]

# 6. Zakres wiekowy (opcjonalnie, np. 18-80 lat)
df_clean = df_clean[(df_clean['age'] >= 18) & (df_clean['age'] <= 80)]

# 7. Wybierz tylko potrzebne kolum
df_final = df_clean[['sex', 'age', 'time_5km_s', 'time_half_s']].copy()

print(f"Po czyszczeniu: {len(df_final)} rekordów (usunięto {len(df) - len(df_final)})")
print(df_final.describe())
print(df_final.head(10))

Po czyszczeniu: 17730 rekordów (usunięto 4227)
                age    time_5km_s   time_half_s
count  17730.000000  17730.000000  17730.000000
mean      39.651043   1670.293739   7393.291483
std       10.235635    245.174173   1213.742594
min       18.000000      0.000000   3843.000000
25%       32.000000   1499.000000   6543.000000
50%       39.500000   1658.000000   7240.000000
75%       46.000000   1831.000000   8132.000000
max       80.000000   2960.000000  12512.000000
  sex   age  time_5km_s  time_half_s
0   M  32.0       877.0       3899.0
1   M  38.0       888.0       3983.0
2   M  28.0       946.0       4104.0
3   M  36.0       971.0       4216.0
4   M  29.0       972.0       4227.0
5   M  41.0       969.0       4234.0
6   M  25.0       937.0       4278.0
7   M  35.0       990.0       4302.0
8   M  23.0      1030.0       4456.0
9   M  32.0      1013.0       4462.0


In [5]:
from io import StringIO

# Zapisz do CSV w pamięci
csv_buffer = StringIO()
df_final.to_csv(csv_buffer, index=False)

# Upload do Spaces
client.put_object(
    Bucket=BUCKET_NAME,
    Key='data/processed/halfmarathon_clean.csv',
    Body=csv_buffer.getvalue().encode('utf-8'),
    ContentType='text/csv'
)

print("Plik halfmarathon_clean.csv zapisany w Spaces!")

Plik halfmarathon_clean.csv zapisany w Spaces!
